In [2]:
from datetime import datetime
import os
from torchinfo import summary
from going_modular.going_modular import data_setup
from going_modular.going_modular.utils import save_model
from going_modular.going_modular.engine import train_step, test_step
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import nn
from tqdm.auto import tqdm
from typing import Dict, List

from torch.utils.tensorboard import SummaryWriter

In [3]:
#  available cuda and def seed
dev = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(42)

In [3]:
# get dir data
train_dir = "/app/pytorch_zero/data/pizza_steak_sushi/train_10/"
test_dir = "/app/pytorch_zero/data/pizza_steak_sushi/test/"

In [4]:
# tranform manual data
# # normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
# #                                  std=[0.229, 0.224, 0.225])

# # manual_tranform = transforms.Compose([
# #     transforms.Resize(size=(224,224)),
# #     transforms.ToTensor(),
# #     normalize
# #     ])

# tranform auto data
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_tranform = weights.transforms()
auto_tranform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [5]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir, test_dir=test_dir,
                                                                               transform=auto_tranform, batch_size=32)

In [6]:
# freezing model
set_seed(42)

# model
model = torchvision.models.efficientnet_b0(weights=weights).to(dev)

for param in model.features.parameters():
    param.requires_grad = False # Change trainable

model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [7]:
# Change classifier 
set_seed(42)

model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names), bias=True).to(dev))
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [9]:
# Track metricks
writer= SummaryWriter()

def train(model: nn.Module, train_dataloader: DataLoader, 
          test_dataloader: DataLoader, optimizer: torch.optim.Optimizer,
          loss_fn: nn.Module,epochs: int, device: torch.device) -> Dict[str, List]:
    
    # Create empty results dictionary
    results = {"train_loss": [], "train_acc": [],
               "test_loss": [], "test_acc": []}
    
    model.to(device)
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model, dataloader=train_dataloader,
                                          loss_fn=loss_fn, optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model, dataloader=test_dataloader,
                                        loss_fn=loss_fn, device=device)

        print( f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}" )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

# Tensorboard Track
        writer.add_scalars(main_tag="Loss", 
                          tag_scalar_dict={"train_loss":train_loss, "test_loss":test_loss},
                          global_step=epoch )
        
        writer.add_scalars(main_tag="Accuracy",
                           tag_scalar_dict={"train_acc":train_acc, "test_acc":test_acc},
                           global_step=epoch)
        
        writer.add_graph(model=model, input_to_model=torch.randn(32,3,224,224).to(dev))

        writer.close()
        
    return results

In [10]:
# train model
set_seed(42)

results = train(model=model, train_dataloader=train_dataloader, 
                       test_dataloader=test_dataloader, optimizer=optimizer,
                       loss_fn=loss_fn, epochs=5, device=dev)
results

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0929 | train_acc: 0.4023 | test_loss: 0.9125 | test_acc: 0.5502


 20%|██        | 1/5 [00:03<00:12,  3.11s/it]

Epoch: 2 | train_loss: 0.8966 | train_acc: 0.6562 | test_loss: 0.7839 | test_acc: 0.8561


 40%|████      | 2/5 [00:05<00:07,  2.47s/it]

...

Epoch: 4 | train_loss: 0.6787 | train_acc: 0.7305 | test_loss: 0.6697 | test_acc: 0.8258


 80%|████████  | 4/5 [00:09<00:02,  2.17s/it]

Epoch: 5 | train_loss: 0.7066 | train_acc: 0.7188 | test_loss: 0.6738 | test_acc: 0.7737


100%|██████████| 5/5 [00:11<00:00,  2.25s/it]


{'train_loss': [1.0929415002465248,
  0.8966035023331642,
  0.8045000210404396,
  0.6786554530262947,
  0.7066160514950752],
 'train_acc': [0.40234375, 0.65625, 0.7421875, 0.73046875, 0.71875],
 'test_loss': [0.912543257077535,
  0.783941368261973,
  0.671596089998881,
  0.6696507334709167,
  0.673759659131368],
 'test_acc': [0.5501893939393939,
  0.8560606060606061,
  0.8863636363636364,
  0.8257575757575758,
  0.7736742424242425]}

In [11]:
# View model track in tensorboard, open port 6006 and share
# %load_ext tensorboard
# %tensorboard --logdir runs

## Create helper function build SummaryWriter()
def create_writer(experiment_name:str, model_name:str, extra:str=None):
    
    timestamp = datetime.now().strftime("%d-%m-%Y")
    
    if extra:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    print(f"INFO: Created SummaryWriter, saving to: {log_dir}..." )
    return SummaryWriter(log_dir=log_dir)

In [12]:
# Create an example writer
example_writer = create_writer(experiment_name="data_10_percent", model_name="effnetb0", extra="5_pochs")

INFO: Created SummaryWriter, saving to: runs/22-02-2024/data_10_percent/effnetb0/5_pochs...


In [13]:
# Update the train and include SummaryWriter
def train(model: nn.Module, train_dataloader: DataLoader, test_dataloader: DataLoader, 
          optimizer: torch.optim.Optimizer, loss_fn: nn.Module,
          epochs: int, writer: SummaryWriter, device: torch.device=dev):

    results = {"train_loss": [], "train_acc": [],
               "test_loss": [], "test_acc": [] }
    
    for epoch in tqdm(range(epochs)):
        
        train_loss, train_acc = train_step(model=model, dataloader=train_dataloader,
                                          loss_fn=loss_fn, optimizer=optimizer,
                                          device=device)
        
        test_loss, test_acc = test_step(model=model, dataloader=test_dataloader,
                                        loss_fn=loss_fn, device=device)

        print( f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | " f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | " f"test_acc: {test_acc:.4f}" )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        if writer:
            writer.add_scalars(main_tag="Loss", 
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                                                global_step=epoch)
            writer.add_scalars(main_tag="Accuracy", 
                               tag_scalar_dict={"train_acc": train_acc, 
                                                "test_acc": test_acc}, 
                                                global_step=epoch)
            writer.close()
        else:
          return results

## Setting up a series of modelling experiments (one 5 epochs and other 10 epochs)


In [14]:
# get data 10% and 20% percent
train_dir_10 = "/app/pytorch_zero/data/pizza_steak_sushi/train_10/"
train_dir_20 = "/app/pytorch_zero/data/pizza_steak_sushi/train_20/"

test_dir = "/app/pytorch_zero/data/pizza_steak_sushi/test/"

In [15]:
# # get weights, MANUAL
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], # values per colour channel [red, green, blue]
#                                  std=[0.229, 0.224, 0.225]) # values per colour channel [red, green, blue]

# simple_transform = transforms.Compose([ 
#     transforms.Resize((224, 224)), 
#     transforms.ToTensor(), 
#     normalize ])

# # get weights AUTO
weights_b0 = torchvision.models.EfficientNet_B0_Weights.DEFAULT
transform_b0 = weights_b0.transforms()

weights_b2 = torchvision.models.EfficientNet_B2_Weights.DEFAULT
transform_b2 = weights_b2.transforms()

In [16]:
# Create 10% and 20% trainnig and tes Dataloader
BATCH_SIZE = 32

train_dataloader_10, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_10, test_dir=test_dir,
                                                                                     transform=transform_b0, batch_size=BATCH_SIZE)

train_dataloader_20, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_20, test_dir=test_dir,
                                                                                     transform=transform_b2, batch_size=BATCH_SIZE)

In [17]:
# create feature extraxtor model
model_b0 = torchvision.models.efficientnet_b0( weights=weights_b0).to(dev)
model_b2 = torchvision.models.efficientnet_b2( weights=weights_b2).to(dev)

In [30]:
# Create def update classifier model, Freezing the base model and change the output layer
def update_output_classifier(in_features, out_features,model, dropout, name,
                             inplace=True, bias=True, dev="cuda", seed=42):

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    for param in model.features.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=dropout, inplace=inplace),
        nn.Linear(in_features=in_features, out_features=out_features, bias=bias) ).to(dev)
    
    model.name = name
    return model

In [31]:
# get values for def update_output_classifier
print(f"Classifier model_b0:{model_b0.classifier}\nClassifier model_b2: {model_b2.classifier}")

Classifier model_b0:Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)
Classifier model_b2: Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1408, out_features=3, bias=True)
)


In [32]:
# update output layer models
model_b0_update = update_output_classifier( model=model_b0, in_features=1280, dropout=0.2, 
                                            out_features=len(class_names), name="model_b0_update" )
model_b2_update = update_output_classifier( model=model_b2, in_features=1408, dropout=0.3,
                                            out_features=len(class_names), name="model_b2_update")

list_model = list([model_b0_update, model_b2_update])
for i in range(len(list_model)):
    print(f"Model {i} Classifier update:{list_model[i].classifier}\n")

Model 0 Classifier update:Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)

Model 1 Classifier update:Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1408, out_features=3, bias=True)
)



In [24]:
# 1. Create epochs list
num_epochs = [5, 10]

# 2. Create models list (need to create a new model for each experiment)
models = ["model_b0_update", "model_b2_update"]

# 3. Create dataloaders dictionary for various dataloaders
train_dataloaders = {"data_10_percent": train_dataloader_10,
                     "data_20_percent": train_dataloader_20}

In [36]:
%%time
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Keep track of experiment numbers
experiment_number = 0

for dataloader_name, train_dataloader in train_dataloaders.items():
    for epochs in num_epochs: 
        for model_name in models:

            # Create information print outs
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")  

            # Select the model
            if model_name == "model_b0_update":
                model = model_b0_update
            else:
                model = model_b2_update
            
            # Create a new loss and optimizer for every model
            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

            train( model=model, train_dataloader=train_dataloader,
                   test_dataloader=test_dataloader, optimizer=optimizer,
                   loss_fn=loss_fn, epochs=epochs, device=dev,
                   writer=create_writer(experiment_name=dataloader_name,
                                        model_name=model_name,
                                        extra=f"{epochs}_epochs"))
            
            # Save the model to file so we can get back the best model
            save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model( model=model, target_dir="models", model_name=save_filepath)
            print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: model_b0_update
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
INFO: Created SummaryWriter, saving to: runs/22-02-2024/data_10_percent/model_b0_update/5_epochs...


 20%|██        | 1/5 [00:01<00:04,  1.24s/it]

Epoch: 1 | train_loss: 1.0929 | train_acc: 0.4023 | test_loss: 0.9260 | test_acc: 0.4877


 40%|████      | 2/5 [00:02<00:03,  1.19s/it]

Epoch: 2 | train_loss: 0.8702 | train_acc: 0.7773 | test_loss: 0.7976 | test_acc: 0.7936


...

Epoch: 8 | train_loss: 0.2023 | train_acc: 0.9563 | test_loss: 0.2883 | test_acc: 0.9176


 90%|█████████ | 9/10 [00:16<00:01,  1.77s/it]

Epoch: 9 | train_loss: 0.2248 | train_acc: 0.9563 | test_loss: 0.3027 | test_acc: 0.9072


100%|██████████| 10/10 [00:17<00:00,  1.78s/it]

Epoch: 10 | train_loss: 0.2162 | train_acc: 0.9208 | test_loss: 0.2809 | test_acc: 0.8968
[INFO] Saving model to: models/07_model_b2_update_data_20_percent_10_epochs.pth
--------------------------------------------------

CPU times: user 26.9 s, sys: 28.1 s, total: 55 s
Wall time: 1min 27s


In [44]:
# Visualize within Tensorboard
%load_ext tensorboard
%tensorboard --logdir runs


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 7574), started 2 days, 12:02:01 ago. (Use '!kill 7574' to kill it.)

In [19]:
# 17